IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\M6\FAULT_M6_5


In [3]:
guasto=1

VOLO m6 FAULT 5%

In [4]:
rcou_m6_fault5 = pd.read_csv("RCOU.csv")
rcou_m6_fault5 = rcou_m6_fault5.astype("float64")
rcou_m6_fault5 = rcou_m6_fault5.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)

potenza = 1350

rcou_m6_fault5 = rcou_m6_fault5[((rcou_m6_fault5['C9'] >= potenza) & (rcou_m6_fault5['C10'] >= potenza) & (rcou_m6_fault5['C11'] >= potenza) & (rcou_m6_fault5['C12'] >= potenza) & (rcou_m6_fault5['C13'] >= potenza) & (rcou_m6_fault5['C14']>= potenza))]
rcou_m6_fault5=rcou_m6_fault5.reset_index(drop=True)
display(rcou_m6_fault5)

,TimeUS,C9,C10,C11,C12,C13,C14
0,67658101.0,1396.0,1412.0,1424.0,1383.0,1411.0,1397.0
1,67668340.0,1449.0,1369.0,1409.0,1412.0,1443.0,1376.0
2,67681963.0,1456.0,1378.0,1418.0,1418.0,1452.0,1382.0
3,67702125.0,1476.0,1381.0,1458.0,1401.0,1497.0,1354.0
4,67712889.0,1372.0,1494.0,1500.0,1365.0,1438.0,1435.0
...,...,...,...,...,...,...,...
4875,125263513.0,1623.0,1385.0,1616.0,1395.0,1566.0,1459.0
4876,125280870.0,1598.0,1400.0,1542.0,1470.0,1471.0,1540.0
4877,125291537.0,1570.0,1417.0,1577.0,1408.0,1491.0,1505.0
4878,125301650.0,1542.0,1426.0,1565.0,1397.0,1463.0,1509.0


In [5]:
min=rcou_m6_fault5['TimeUS'][0]
max=rcou_m6_fault5['TimeUS'][len(rcou_m6_fault5)-1]
print(max)
print(min)

125327720.0
67658101.0


In [6]:
xkf1_m6_fault5 = pd.read_csv("XKF1_0.csv")
xkf1_m6_fault5 = xkf1_m6_fault5.astype("float64")
xkf1_m6_fault5 = xkf1_m6_fault5.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m6_fault5 = xkf1_m6_fault5[((xkf1_m6_fault5['TimeUS'] >= min) & (xkf1_m6_fault5['TimeUS'] <= max	))]
xkf1_m6_fault5 = xkf1_m6_fault5.reset_index(drop=True)
print(xkf1_m6_fault5)

           TimeUS  Roll  Pitch    Yaw
0      67666069.0  0.54  -1.23  28.83
1      67679934.0  0.55  -1.25  28.82
2      67689697.0  0.66  -1.31  28.83
3      67700022.0  0.69  -1.36  28.82
4      67710276.0  0.63  -1.35  28.83
...           ...   ...    ...    ...
5214  125278300.0 -2.21   2.07  12.35
5215  125289227.0 -2.17   1.91  12.25
5216  125299633.0 -2.14   1.77  12.11
5217  125309609.0 -2.09   1.60  11.97
5218  125325522.0 -1.98   1.23  11.72

[5219 rows x 4 columns]


In [7]:
att_m6_fault5 = pd.read_csv("ATT.csv")
att_m6_fault5 = att_m6_fault5.astype("float64")
att_m6_fault5 = att_m6_fault5.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m6_fault5 = att_m6_fault5[((att_m6_fault5['TimeUS'] >= min) & (att_m6_fault5['TimeUS'] <= max	))]
att_m6_fault5=att_m6_fault5.reset_index(drop=True)
print(att_m6_fault5)

           TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0      67667560.0     0.11  0.54      0.42  -1.23   28.71  28.83
1      67681139.0     0.10  0.55      0.43  -1.25   28.71  28.82
2      67691201.0     0.09  0.66      0.45  -1.31   28.71  28.83
3      67701334.0     0.08  0.69      0.47  -1.36   28.71  28.82
4      67712253.0     0.07  0.63      0.48  -1.35   28.71  28.83
...           ...      ...   ...       ...    ...     ...    ...
5214  125280024.0    -1.27 -2.21      1.24   2.07   12.32  12.35
5215  125290917.0    -1.27 -2.17      1.24   1.91   12.32  12.25
5216  125300929.0    -1.29 -2.14      1.25   1.77   12.32  12.11
5217  125311929.0    -1.32 -2.09      1.28   1.60   12.32  11.97
5218  125327033.0    -1.36 -1.98      1.30   1.23   12.32  11.72

[5219 rows x 7 columns]


In [8]:
from numpy import empty

if len(att_m6_fault5)>len(xkf1_m6_fault5):
    to_add=att_m6_fault5[len(xkf1_m6_fault5):]
    att_m6_fault5=att_m6_fault5[:len(xkf1_m6_fault5)]

if len(xkf1_m6_fault5)>len(att_m6_fault5):
    to_add=xkf1_m6_fault5[len(att_m6_fault5):]

for idx,i in enumerate(att_m6_fault5['Roll']):
    if(xkf1_m6_fault5['Roll'][idx] != i):
        att_m6_fault5['Roll'][idx] = ((att_m6_fault5['Roll'][idx] + xkf1_m6_fault5['Roll'][idx])/2)

for idx,i in enumerate(att_m6_fault5['Pitch']):
    if(xkf1_m6_fault5['Pitch'][idx] != i):
        att_m6_fault5['Pitch'][idx] = ((att_m6_fault5['Pitch'][idx] + xkf1_m6_fault5['Pitch'][idx])/2)

for idx,i in enumerate(att_m6_fault5['Yaw']):
    if(xkf1_m6_fault5['Yaw'][idx] != i):
        att_m6_fault5['Yaw'][idx] = ((att_m6_fault5['Yaw'][idx] + xkf1_m6_fault5['Yaw'][idx])/2)
if 'to_add' in globals():
    att_m6_fault5=pd.concat([att_m6_fault5,to_add])

print(att_m6_fault5)

           TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0      67667560.0     0.11  0.54      0.42  -1.23   28.71  28.83
1      67681139.0     0.10  0.55      0.43  -1.25   28.71  28.82
2      67691201.0     0.09  0.66      0.45  -1.31   28.71  28.83
3      67701334.0     0.08  0.69      0.47  -1.36   28.71  28.82
4      67712253.0     0.07  0.63      0.48  -1.35   28.71  28.83
...           ...      ...   ...       ...    ...     ...    ...
5214  125280024.0    -1.27 -2.21      1.24   2.07   12.32  12.35
5215  125290917.0    -1.27 -2.17      1.24   1.91   12.32  12.25
5216  125300929.0    -1.29 -2.14      1.25   1.77   12.32  12.11
5217  125311929.0    -1.32 -2.09      1.28   1.60   12.32  11.97
5218  125327033.0    -1.36 -1.98      1.30   1.23   12.32  11.72

[5219 rows x 7 columns]


In [9]:
'''#elettricita' che scorre nel motore considerato, al momento ne consideriamo solo 1
esc_0_m6_fault5 = pd.read_csv("ESC_0.csv")
esc_0_m6_fault5 = esc_0_m6_fault5.astype("float64")
esc_0_m6_fault5 = esc_0_m6_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_0_m6_fault5 = esc_0_m6_fault5[((esc_0_m6_fault5['TimeUS'] >= min) & (esc_0_m6_fault5['TimeUS'] <= max))]
esc_0_m6_fault5=esc_0_m6_fault5.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})

esc_1_m6_fault5 = pd.read_csv("ESC_1.csv")
esc_1_m6_fault5 = esc_1_m6_fault5.astype("float64")
esc_1_m6_fault5 = esc_1_m6_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_1_m6_fault5 = esc_1_m6_fault5[((esc_1_m6_fault5['TimeUS'] >= min) & (esc_1_m6_fault5['TimeUS'] <= max))]
esc_1_m6_fault5=esc_1_m6_fault5.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})

esc_2_m6_fault5 = pd.read_csv("ESC_2.csv")
esc_2_m6_fault5 = esc_2_m6_fault5.astype("float64")
esc_2_m6_fault5 = esc_2_m6_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_2_m6_fault5 = esc_2_m6_fault5[((esc_2_m6_fault5['TimeUS'] >= min) & (esc_2_m6_fault5['TimeUS'] <= max))]
esc_2_m6_fault5=esc_2_m6_fault5.rename(columns={"RPM": "RPM(ESC_2)", "Curr": "Curr(ESC_2)"})

esc_3_m6_fault5 = pd.read_csv("ESC_3.csv")
esc_3_m6_fault5 = esc_3_m6_fault5.astype("float64")
esc_3_m6_fault5 = esc_3_m6_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_3_m6_fault5 = esc_3_m6_fault5[((esc_3_m6_fault5['TimeUS'] >= min) & (esc_3_m6_fault5['TimeUS'] <= max))]
esc_3_m6_fault5=esc_3_m6_fault5.rename(columns={"RPM": "RPM(ESC_3)", "Curr": "Curr(ESC_3)"})

esc_4_m6_fault5 = pd.read_csv("ESC_4.csv")
esc_4_m6_fault5 = esc_4_m6_fault5.astype("float64")
esc_4_m6_fault5 = esc_4_m6_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_4_m6_fault5 = esc_4_m6_fault5[((esc_4_m6_fault5['TimeUS'] >= min) & (esc_4_m6_fault5['TimeUS'] <= max))]
esc_4_m6_fault5=esc_4_m6_fault5.rename(columns={"RPM": "RPM(ESC_4)", "Curr": "Curr(ESC_4)"})

esc_5_m6_fault5 = pd.read_csv("ESC_5.csv")
esc_5_m6_fault5 = esc_5_m6_fault5.astype("float64")
esc_5_m6_fault5 = esc_5_m6_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_5_m6_fault5 = esc_5_m6_fault5[((esc_5_m6_fault5['TimeUS'] >= min) & (esc_5_m6_fault5['TimeUS'] <= max))]
esc_5_m6_fault5=esc_5_m6_fault5.rename(columns={"RPM": "RPM(ESC_5)", "Curr": "Curr(ESC_5)"})'''

'#elettricita\' che scorre nel motore considerato, al momento ne consideriamo solo 1\nesc_0_m6_fault5 = pd.read_csv("ESC_0.csv")\nesc_0_m6_fault5 = esc_0_m6_fault5.astype("float64")\nesc_0_m6_fault5 = esc_0_m6_fault5.drop([\'LineNo\',\'Instance\', \'RawRPM\', \'Volt\', \'Temp\', \'CTot\', \'MotTemp\', \'Err\'], axis=1)\nesc_0_m6_fault5 = esc_0_m6_fault5[((esc_0_m6_fault5[\'TimeUS\'] >= min) & (esc_0_m6_fault5[\'TimeUS\'] <= max))]\nesc_0_m6_fault5=esc_0_m6_fault5.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})\n\nesc_1_m6_fault5 = pd.read_csv("ESC_1.csv")\nesc_1_m6_fault5 = esc_1_m6_fault5.astype("float64")\nesc_1_m6_fault5 = esc_1_m6_fault5.drop([\'LineNo\',\'Instance\', \'RawRPM\', \'Volt\', \'Temp\', \'CTot\', \'MotTemp\', \'Err\'], axis=1)\nesc_1_m6_fault5 = esc_1_m6_fault5[((esc_1_m6_fault5[\'TimeUS\'] >= min) & (esc_1_m6_fault5[\'TimeUS\'] <= max))]\nesc_1_m6_fault5=esc_1_m6_fault5.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})\n\nesc_2_m6_fault5 = pd.r

In [10]:
#valori di accelerazione e del giroscopio
imu_0_m6_fault5 = pd.read_csv("IMU_0.csv")
imu_0_m6_fault5 = imu_0_m6_fault5.astype("float64")
imu_0_m6_fault5 = imu_0_m6_fault5.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m6_fault5 = imu_0_m6_fault5[((imu_0_m6_fault5['TimeUS'] >= min) & (imu_0_m6_fault5['TimeUS'] <= max))]

imu_1_m6_fault5 = pd.read_csv("IMU_1.csv")
imu_1_m6_fault5 = imu_1_m6_fault5.astype("float64")
imu_1_m6_fault5 = imu_1_m6_fault5.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m6_fault5 = imu_1_m6_fault5[((imu_1_m6_fault5['TimeUS'] >= min) & (imu_1_m6_fault5['TimeUS'] <= max))]

imu_2_m6_fault5 = pd.read_csv("IMU_2.csv")
imu_2_m6_fault5 = imu_2_m6_fault5.astype("float64")
imu_2_m6_fault5 = imu_2_m6_fault5.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m6_fault5 = imu_2_m6_fault5[((imu_2_m6_fault5['TimeUS'] >= min) & (imu_2_m6_fault5['TimeUS'] <= max))]

imu_m6_fault5 = pd.concat((imu_0_m6_fault5, imu_1_m6_fault5, imu_2_m6_fault5))
imu_m6_fault5=imu_m6_fault5.groupby(imu_m6_fault5.TimeUS, as_index=False).mean()

display(imu_m6_fault5)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,67659762.0,-0.138047,-0.038018,0.001737,0.033579,0.086641,-9.665883
1,67662879.0,-0.011009,-0.058367,-0.001887,0.068102,0.299002,-9.464789
2,67665259.0,0.100030,-0.062974,-0.002615,0.024549,0.329164,-9.613195
3,67667399.0,0.097606,-0.021730,0.007218,-0.066825,0.120736,-9.898083
4,67670078.0,-0.066935,0.001945,0.007067,-0.104688,0.004512,-10.003252
...,...,...,...,...,...,...,...
19887,125308469.0,0.163667,-0.395403,-0.270902,1.135757,1.445938,-12.786887
19888,125311848.0,0.078180,-0.325693,-0.265427,0.796094,1.550297,-12.949453
19889,125314499.0,0.040967,-0.191538,-0.240225,0.893079,1.877303,-12.723017
19890,125316963.0,0.154693,-0.385259,-0.279038,1.115688,1.947423,-12.658253


In [11]:
#sincronizzazione dei tempi tra att e imu_2
m6_fault5 = pd.merge_ordered(imu_m6_fault5,att_m6_fault5)
m6_fault5=m6_fault5.fillna(method="ffill").fillna(method="bfill")
#sincronizzazione dei tempi tra la tabella ottenuta precedentemente e esc_0 (corrente che passa nel motore)
'''m6_fault5 = pd.merge_ordered(m6_fault5, esc_0_m6_fault5)
m6_fault5 = pd.merge_ordered(m6_fault5, esc_1_m6_fault5)
m6_fault5 = pd.merge_ordered(m6_fault5, esc_2_m6_fault5)
m6_fault5 = pd.merge_ordered(m6_fault5, esc_3_m6_fault5)
m6_fault5 = pd.merge_ordered(m6_fault5, esc_4_m6_fault5)
m6_fault5 = pd.merge_ordered(m6_fault5, esc_5_m6_fault5)
m6_fault5=m6_fault5.fillna(method="ffill").fillna(method="bfill")'''
m6_fault5 = pd.merge_ordered(m6_fault5, rcou_m6_fault5)
m6_fault5=m6_fault5.fillna(method="ffill").fillna(method="bfill")

In [12]:
m6_fault5["TimeUS"] = m6_fault5["TimeUS"] - m6_fault5.iloc[0]["TimeUS"]
m6_fault5["TimeUS"] = m6_fault5["TimeUS"].astype(int)
m6_fault5["Guasto"] = guasto

In [13]:
last_cell = m6_fault5.iloc[-1, m6_fault5.columns.get_loc('TimeUS')]
print(last_cell)
to_be=[]
for i in range(0, last_cell,2857):
    to_be.append(i)
df = pd.DataFrame(to_be, columns = ['TimeUS'])


m6_fault5_final = pd.merge_ordered(m6_fault5,df)
m6_fault5_final=m6_fault5_final.fillna(method="ffill").fillna(method="bfill")
print(m6_fault5_final)


m6_fault5_final = m6_fault5_final[m6_fault5_final.TimeUS.isin(to_be)]

m6_fault5_final

57669619
         TimeUS      GyrX      GyrY      GyrZ      AccX      AccY       AccZ  \
0             0 -0.138047 -0.038018  0.001737  0.033579  0.086641  -9.665883   
1          1661 -0.138047 -0.038018  0.001737  0.033579  0.086641  -9.665883   
2          2857 -0.138047 -0.038018  0.001737  0.033579  0.086641  -9.665883   
3          4778 -0.011009 -0.058367 -0.001887  0.068102  0.299002  -9.464789   
4          5714 -0.011009 -0.058367 -0.001887  0.068102  0.299002  -9.464789   
...         ...       ...       ...       ...       ...       ...        ...   
50162  57665688  0.154693 -0.385259 -0.279038  1.115688  1.947423 -12.658253   
50163  57668545  0.154693 -0.385259 -0.279038  1.115688  1.947423 -12.658253   
50164  57668767  0.036664 -0.298780 -0.266965  0.743121  2.141147 -12.724967   
50165  57668932  0.036664 -0.298780 -0.266965  0.743121  2.141147 -12.724967   
50166  57669619  0.036664 -0.298780 -0.266965  0.743121  2.141147 -12.724967   

       DesRoll  Roll  DesPitch

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,Pitch,DesYaw,Yaw,C9,C10,C11,C12,C13,C14,Guasto
0,0,-0.138047,-0.038018,0.001737,0.033579,0.086641,-9.665883,0.11,0.54,0.42,-1.23,28.71,28.83,1396.0,1412.0,1424.0,1383.0,1411.0,1397.0,1.0
2,2857,-0.138047,-0.038018,0.001737,0.033579,0.086641,-9.665883,0.11,0.54,0.42,-1.23,28.71,28.83,1396.0,1412.0,1424.0,1383.0,1411.0,1397.0,1.0
4,5714,-0.011009,-0.058367,-0.001887,0.068102,0.299002,-9.464789,0.11,0.54,0.42,-1.23,28.71,28.83,1396.0,1412.0,1424.0,1383.0,1411.0,1397.0,1.0
6,8571,0.100030,-0.062974,-0.002615,0.024549,0.329164,-9.613195,0.11,0.54,0.42,-1.23,28.71,28.83,1396.0,1412.0,1424.0,1383.0,1411.0,1397.0,1.0
10,11428,0.097606,-0.021730,0.007218,-0.066825,0.120736,-9.898083,0.11,0.54,0.42,-1.23,28.71,28.83,1449.0,1369.0,1409.0,1412.0,1443.0,1376.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50158,57657117,0.040967,-0.191538,-0.240225,0.893079,1.877303,-12.723017,-1.32,-2.09,1.28,1.60,12.32,11.97,1542.0,1426.0,1565.0,1397.0,1463.0,1509.0,1.0
50160,57659974,0.154693,-0.385259,-0.279038,1.115688,1.947423,-12.658253,-1.32,-2.09,1.28,1.60,12.32,11.97,1542.0,1426.0,1565.0,1397.0,1463.0,1509.0,1.0
50161,57662831,0.154693,-0.385259,-0.279038,1.115688,1.947423,-12.658253,-1.32,-2.09,1.28,1.60,12.32,11.97,1542.0,1426.0,1565.0,1397.0,1463.0,1509.0,1.0
50162,57665688,0.154693,-0.385259,-0.279038,1.115688,1.947423,-12.658253,-1.32,-2.09,1.28,1.60,12.32,11.97,1542.0,1426.0,1565.0,1397.0,1463.0,1509.0,1.0


In [14]:
from datetime import timedelta
m6_fault5_final=m6_fault5_final.reset_index(drop=True)
m6_fault5_final['TimeUS'] = pd.to_datetime(m6_fault5_final['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m6_fault5_final)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,Pitch,DesYaw,Yaw,C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,-0.138047,-0.038018,0.001737,0.033579,0.086641,-9.665883,0.11,0.54,0.42,-1.23,28.71,28.83,1396.0,1412.0,1424.0,1383.0,1411.0,1397.0,1.0
1,00:00:00.002857,-0.138047,-0.038018,0.001737,0.033579,0.086641,-9.665883,0.11,0.54,0.42,-1.23,28.71,28.83,1396.0,1412.0,1424.0,1383.0,1411.0,1397.0,1.0
2,00:00:00.005714,-0.011009,-0.058367,-0.001887,0.068102,0.299002,-9.464789,0.11,0.54,0.42,-1.23,28.71,28.83,1396.0,1412.0,1424.0,1383.0,1411.0,1397.0,1.0
3,00:00:00.008571,0.100030,-0.062974,-0.002615,0.024549,0.329164,-9.613195,0.11,0.54,0.42,-1.23,28.71,28.83,1396.0,1412.0,1424.0,1383.0,1411.0,1397.0,1.0
4,00:00:00.011428,0.097606,-0.021730,0.007218,-0.066825,0.120736,-9.898083,0.11,0.54,0.42,-1.23,28.71,28.83,1449.0,1369.0,1409.0,1412.0,1443.0,1376.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20181,00:00:57.657117,0.040967,-0.191538,-0.240225,0.893079,1.877303,-12.723017,-1.32,-2.09,1.28,1.60,12.32,11.97,1542.0,1426.0,1565.0,1397.0,1463.0,1509.0,1.0
20182,00:00:57.659974,0.154693,-0.385259,-0.279038,1.115688,1.947423,-12.658253,-1.32,-2.09,1.28,1.60,12.32,11.97,1542.0,1426.0,1565.0,1397.0,1463.0,1509.0,1.0
20183,00:00:57.662831,0.154693,-0.385259,-0.279038,1.115688,1.947423,-12.658253,-1.32,-2.09,1.28,1.60,12.32,11.97,1542.0,1426.0,1565.0,1397.0,1463.0,1509.0,1.0
20184,00:00:57.665688,0.154693,-0.385259,-0.279038,1.115688,1.947423,-12.658253,-1.32,-2.09,1.28,1.60,12.32,11.97,1542.0,1426.0,1565.0,1397.0,1463.0,1509.0,1.0


## Analisi nel tempo e in frequenza

In [15]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

## Funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza

In [16]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})'])
    for i in range(int(len(m6_fault5_final)/350)):
        V1=m6_fault5_final[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        #for k in range(len(peaks_index)):
        #[print("%4.4f    \t %3.4f" %(freq[peaks_index[i]], properties['peak_heights'][i])) for i in range(len(peaks_index))]
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2}
        tab = tab.astype("float64")
        tab=tab.append(val,ignore_index=True)
    return tab

In [17]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
'''rpm_ESC_0=time_freq_feat('RPM(ESC_0)')
curr_ESC_0=time_freq_feat('Curr(ESC_0)')
rpm_ESC_1=time_freq_feat('RPM(ESC_1)')
curr_ESC_1=time_freq_feat('Curr(ESC_1)')
rpm_ESC_2=time_freq_feat('RPM(ESC_2)')
curr_ESC_2=time_freq_feat('Curr(ESC_2)')
rpm_ESC_3=time_freq_feat('RPM(ESC_3)')
curr_ESC_3=time_freq_feat('Curr(ESC_3)')
rpm_ESC_4=time_freq_feat('RPM(ESC_4)')
curr_ESC_4=time_freq_feat('Curr(ESC_4)')
rpm_ESC_5=time_freq_feat('RPM(ESC_5)')
curr_ESC_5=time_freq_feat('Curr(ESC_5)')'''
c_9 =time_freq_feat('C9')
c_10 =time_freq_feat('C10')
c_11 =time_freq_feat('C11')
c_12 =time_freq_feat('C12')
c_13 =time_freq_feat('C13')
c_14 =time_freq_feat('C14')
#merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,rpm_ESC_0,curr_ESC_0,rpm_ESC_1,curr_ESC_1,rpm_ESC_2,curr_ESC_2,rpm_ESC_3,curr_ESC_3,rpm_ESC_4,curr_ESC_4,rpm_ESC_5,curr_ESC_5,c_9,c_10,c_11,c_12,c_13,c_14]
merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,c_9,c_10,c_11,c_12,c_13,c_14]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
df_merged["Guasto"] = guasto

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
display(df_merged)

c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_34104\2376279633.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_34104\2376279633.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards 

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),<>(GyrY),var(GyrY),kurt(GyrY),RMS(GyrY),freq1(GyrY),freq2(GyrY),amp1(GyrY),amp2(GyrY),<>(GyrZ),var(GyrZ),kurt(GyrZ),RMS(GyrZ),freq1(GyrZ),freq2(GyrZ),amp1(GyrZ),amp2(GyrZ),<>(AccX),var(AccX),kurt(AccX),RMS(AccX),freq1(AccX),freq2(AccX),amp1(AccX),amp2(AccX),<>(AccY),var(AccY),kurt(AccY),RMS(AccY),freq1(AccY),freq2(AccY),amp1(AccY),amp2(AccY),<>(AccZ),var(AccZ),kurt(AccZ),RMS(AccZ),freq1(AccZ),freq2(AccZ),amp1(AccZ),amp2(AccZ),<>(Roll),var(Roll),kurt(Roll),RMS(Roll),freq1(Roll),freq2(Roll),amp1(Roll),amp2(Roll),<>(Pitch),var(Pitch),kurt(Pitch),RMS(Pitch),freq1(Pitch),freq2(Pitch),amp1(Pitch),amp2(Pitch),<>(Yaw),var(Yaw),kurt(Yaw),RMS(Yaw),freq1(Yaw),freq2(Yaw),amp1(Yaw),amp2(Yaw),<>(C9),var(C9),kurt(C9),RMS(C9),freq1(C9),freq2(C9),amp1(C9),amp2(C9),<>(C10),var(C10),kurt(C10),RMS(C10),freq1(C10),freq2(C10),amp1(C10),amp2(C10),<>(C11),var(C11),kurt(C11),RMS(C11),freq1(C11),freq2(C11),amp1(C11),amp2(C11),<>(C12),var(C12),kurt(C12),RMS(C12),freq1(C12),freq2(C12),amp1(C12),amp2(C12),<>(C13),var(C13),kurt(C13),RMS(C13),freq1(C13),freq2(C13),amp1(C13),amp2(C13),<>(C14),var(C14),kurt(C14),RMS(C14),freq1(C14),freq2(C14),amp1(C14),amp2(C14),Guasto
0,0.0,-0.104704,0.039568,-0.394323,0.224541,47.0,60.0,5.906655,2.975821,0.049461,0.010709,-0.780838,0.114565,89.0,64.0,7.114336,2.432363,0.113753,0.004592,-0.921744,0.132359,74.0,62.0,1.042070,0.564347,0.121023,0.102692,-0.653423,0.342120,94.0,99.0,9.948305,8.087049,0.248250,0.056721,1.614931,0.343784,82.0,66.0,11.316682,7.362393,-10.917529,0.825123,-1.186094,10.955145,0.0,109.0,211.335656,11.131317,0.147009,3.308918,1.399029,1.822389,51.0,67.0,6.119758,4.974693,0.455071,2.012611,-1.776950,1.487940,128.0,126.0,25.944362,1.404946,31.318291,5.017455,-1.315569,31.398065,80.0,106.0,-1.607472,-2.126594,1590.179487,8038.519121,0.902320,1592.697843,55.0,52.0,2025.866334,781.737772,1492.584046,2579.895059,-0.464602,1493.445573,75.0,72.0,2027.925658,860.406772,1577.264957,2951.086740,1.897058,1578.197524,101.0,103.0,1212.556933,950.618403,1510.225071,3855.409198,-0.731468,1511.497334,91.0,94.0,902.585158,870.185263,1542.444444,3219.236190,0.761172,1543.484671,85.0,83.0,2371.532626,881.931952,1546.253561,4234.246952,0.692110,1547.618254,87.0,86.0,1572.189807,1223.458119,1
1,1.0,-0.105454,0.039654,-0.403180,0.225081,49.0,64.0,6.858856,2.863659,0.049235,0.010697,-0.775998,0.114414,93.0,64.0,7.072210,1.878003,0.113680,0.004581,-0.920755,0.132253,77.0,55.0,0.945083,0.590180,0.121187,0.102409,-0.646235,0.341767,103.0,108.0,9.488531,7.862545,0.249103,0.056815,1.602540,0.344537,84.0,66.0,9.540625,6.347245,-10.915809,0.823813,-1.182011,10.953372,0.0,105.0,211.748441,11.097761,0.131364,3.385647,1.366123,1.842087,51.0,68.0,5.646021,5.102631,0.458494,2.011001,-1.774740,1.488457,127.0,125.0,26.642165,1.409184,31.329915,5.050723,-1.318578,31.410188,109.0,85.0,-2.234577,-2.412723,1590.394886,8031.949034,0.913288,1592.910866,67.0,64.0,2016.683527,641.224230,1492.386364,2586.300440,-0.480902,1493.250150,83.0,80.0,1997.885251,992.107494,1577.440341,2953.506402,1.900461,1578.373576,104.0,106.0,1183.855837,916.230405,1510.079545,3851.879694,-0.736878,1511.350777,101.0,100.0,823.491927,797.250913,1542.357955,3212.697714,0.759060,1543.396135,96.0,95.0,2332.924405,860.945917,1546.394886,4229.213991,0.702397,1547.757844,98.0,96.0,1536.565702,1110.540248,1
2,2.0,-0.106199,0.039737,-0.411822,0.225616,50.0,66.0,6.835986,2.706003,0.049009,0.010684,-0.771120,0.114263,95.0,72.0,7.028093,2.110259,0.113607,0.004570,-0.919746,0.132149,78.0,55.0,0.849184,0.657167,0.121349,0.102128,-0.639045,0.341415,98.0,103.0,8.618544,7.592629,0.249950,0.056907,1.590449,0.345283,89.0,93.0,7.239032,6.557950,-10.914100,0.822505,-1.177895,10.951609,0.0,117.0,212.143377,11.048417,0.115807,3.461454,1.329318,1.861467,75.0,66.0,5.281684,4.210199,0.461898,2.009378,-1.772485,1.488971,124.0,122.0,27.343507,1.402084,31.341473,5.083533,-1.321800,31.422238,79.0,105.0,-1.622

In [18]:
path_file = path_file.replace(r"M6\FAULT_M6_5", "")
os.chdir(path_file)
df_merged.to_csv('m6_fault5.csv', index=False)
print(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\
